# ETA_Adjustment

Before you run you must complete the following steps. 

    Change shipping number header to "Logical Tracking ID"

    Dont forget to also update the date (Three of them) before the loops

In [ ]:
import pandas as pd
import pyodbc
import numpy as np
from datetime import datetime, timedelta, date,time
import urllib

## Primary Query

In [ ]:
conn=pyodbc.connect(r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
                    r'DBQ=//cpint_data/data/MERCHANDISING/ReBuyers Shared/Daily Updates - Purchasing.mdb;')
po_out_qry="Select * From [PO Out]"
po_pd=pd.read_sql_query(po_out_qry,conn)

In [ ]:
wkend_pd = pd.read_excel('weekend_Receipts.xlsx',sheetname='Sheet1',index_col=2)
wkend_pd['TRANSACTION_DATE'] = wkend_pd['TRANSACTION_DATE'].apply(lambda x: x.strftime('%Y-%m-%d'))
wkend_pd['DT_IMPORTED'] = wkend_pd['DT_IMPORTED'].apply(lambda x: x.strftime('%Y-%m-%d'))
wkend_pd['Track_ID'] = wkend_pd['PO_NBR'].map(str)
wkend_pd['Track_ID'] = wkend_pd['Track_ID'].astype(str).str[:-2]
wkend_pd['Track_ID'] = wkend_pd['Track_ID'] + '-' + wkend_pd.index
fri_receipts_pd = wkend_pd.loc[wkend_pd['TRANSACTION_DATE'] == '2018-05-18']
wkend_pd.head()

## Past Receipts

In [ ]:
conn_net = pyodbc.connect(r'DRIVER={NetezzaSQL};'
                          r'SERVER=PUREDATA_PRD;'
                          r'PORT=5480;'
                          r'DATABASE=DNA_PUB_INVENTORY;'
                          r'UID=*******;'
                          r'PWD=*********')

In [ ]:
receipts_qry = ('SELECT WMS_RECEIPTS_ALLORG.SHIPMENT_NUMBER, WMS_RECEIPTS_ALLORG.ORG, '
                'WMS_RECEIPTS_ALLORG.TRANSACTION_DATE, WMS_RECEIPTS_ALLORG.VERIFIED, '
                'WMS_RECEIPTS_ALLORG.PO_NBR, WMS_RECEIPTS_ALLORG.SKU, '
                'WMS_RECEIPTS_ALLORG.RECEIVE_QTY, WMS_RECEIPTS_ALLORG.RECEIVE_CARTON,' 
                'WMS_RECEIPTS_ALLORG.DT_IMPORTED '
                'FROM WMS_RECEIPTS_ALLORG '
                "WHERE ((WMS_RECEIPTS_ALLORG.TRANSACTION_DATE)>'3/1/2018' );" 
                )

In [ ]:
# Run Netezza query
wkend_pd=pd.read_sql_query(receipts_qry,conn_net, index_col='SHIPMENT_NUMBER')
wkend_pd['TRANSACTION_DATE'] = wkend_pd['TRANSACTION_DATE'].apply(lambda x: x.strftime('%Y-%m-%d'))
wkend_pd['DT_IMPORTED'] = wkend_pd['DT_IMPORTED'].apply(lambda x: x.strftime('%Y-%m-%d'))
wkend_pd['Track_ID'] = wkend_pd['PO_NBR'].map(str)
wkend_pd['Track_ID'] = wkend_pd['Track_ID'].astype(str).str[:-2]
wkend_pd['Track_ID'] = wkend_pd['Track_ID'] + '-' + wkend_pd.index
fri_receipts_pd = wkend_pd.loc[wkend_pd['TRANSACTION_DATE'] == '2018-05-18']
wkend_pd.head()

# Just for Mondays Data

## Only use if not Monday

In [ ]:
#Just for Mondays Data
conn=pyodbc.connect(r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
                    r'DBQ=//hftdata02/sysdata02/RedCat/Python/ETA_Adjustment/Daily Updates - Purchasing.accdb;')
po_out_qry="Select * From [PO Out]"
po_pd=pd.read_sql_query(po_out_qry,conn)
len(po_pd)

In [ ]:
# so you dont have to keep pulling the data
revert = po_pd

In [ ]:
po_pd = revert

## Container Queries

In [ ]:
cont_hdr_qry = ("Select [Logical Tracking ID],Vessel,Status,ETD_Date, ETA_Date,POL,POD,[Container Size], "
                "Cartons,Location,Container,SCAC,IN_DATE, OUT_DATE,Date_Available, Date_Ordered, [Date_Out Gated], "
                "[Date_1st Arrived], [On Priority], Date_Empty, [Picked up for_Term], Date_Terminated "
                "From [cont hdr]")

cont_hdr_pd=pd.read_sql_query(cont_hdr_qry,conn, index_col='Logical Tracking ID')

cont_dtl_qry = "Select * From [Cont Dtl]"

cont_dtl_pd=pd.read_sql_query(cont_dtl_qry,conn,index_col='Logical Tracking ID')

In [ ]:
cont_pd = cont_dtl_pd.join(cont_hdr_pd, lsuffix='_l', rsuffix='_r')
cont_pd['Track_ID'] = cont_pd['PO Number']+'-'+cont_pd.index
cont_pd.reset_index(inplace=True)
cont_pd['Ship Qty'] = cont_pd[cont_pd['Ship Qty'] > 0]['Ship Qty']
cont_pd = cont_pd[np.isfinite(cont_pd['Ship Qty'])]
cont_pd.reset_index()
cont_pd.set_index('Logical Tracking ID',inplace=True)
cont_pd['ID'] = cont_pd['Item'].map(str)+'-'+cont_pd['PO Number'].map(str)
cont_pd['Ship To Location']='DILLON'
cont_pd.loc[cont_pd['Org']=='MOR','Ship To Location']='MORENO VALLEY'
cont_pd.loc[cont_pd['Org']=='CAM','Ship To Location']='CAMARILLO'
#Filters out all of the lines that were received since the file was last updated. 
cont_pd = cont_pd[~(cont_pd['Track_ID'].isin(wkend_pd['Track_ID']))]

In [ ]:
po_filter_pd=cont_pd[['Item','PO Number','Ship Qty']]
po_filter_pd['ID'] = po_filter_pd['Item'].map(str)+'-'+po_filter_pd['PO Number'].map(str)
po_filter_pd.reset_index(inplace=True)
cont_grouped = po_filter_pd.groupby('ID').sum()
cont_grouped = cont_grouped.rename(columns={'Item':'SKU','Ship Qty':'ship_qty'})

In [ ]:
po_pd['ID'] = po_pd['SKU'].map(str)+'-'+po_pd['Legacy PO NUM'].map(str)
po_pd.set_index('ID',inplace=True)
po_pd = po_pd.join(cont_grouped, how='left', rsuffix='_r')
po_pd['ship_qty'].fillna(0, inplace=True)
po_pd['ship_qty'] = po_pd['ship_qty'].map(int)
po_pd['Adj_Qty_raw']=po_pd['PO Open Qty']-po_pd['ship_qty']
po_pd['Adj_Qty'] = np.where(po_pd["Adj_Qty_raw"] <0,0,po_pd['Adj_Qty_raw'])
#po_pd['Adj_Qty'] = po_pd[po_pd['Adj_Qty'] > 0]['Adj_Qty']
po_pd = po_pd[np.isfinite(po_pd['Adj_Qty'])]
po_pd = po_pd.sort_values('Adj_Qty_raw',ascending=True)
len(po_pd)

In [ ]:
cont_pd.reset_index(inplace=True)
cont_pd=cont_pd[['Logical Tracking ID','Item','PO Number','Ord Qty', 'Ship To Location','Ship Qty','ETD_Date','Supplier Number','ETA_Date','POD']]
cont_pd['FOB-Cost']=0
cont_pd['PO Open Qty'] = cont_pd['Ship Qty'].values
cont_pd = cont_pd.rename(columns={'Item':'SKU','PO Number':'PO Number', 'Ship To Location':'Ship To Location',
                                  'POD':'POD','Ship Qty':'Shipped Qty','ETD_Date':'Approved_Date','Supplier Number':'Vendor No',
                                  'ETA_Date':'WE_Date'})
cont_pd = cont_pd.sort_values('WE_Date',ascending=True)
cont_pd['day_of_week']=cont_pd['WE_Date'].dt.dayofweek
cont_pd.head()

In [ ]:
fri={0:4,1:3,2:2,3:1,4:0,5:6,6:5}
start_date = datetime(2018,5,18) #previous Friday
fri_lst =[]
for i, row in cont_pd.iterrows():
    fri_date = (cont_pd.get_value(i,'WE_Date',takeable=False)
                + timedelta(days=fri[cont_pd.get_value(i,'day_of_week',takeable=False)]))
    if fri_date <= start_date:
        fri_date=start_date
    fri_lst.append(fri_date)
se = pd.Series(fri_lst)    
cont_pd['Fri_date'] = se.values
grouped_df = cont_pd.groupby( [ 'Logical Tracking ID','POD', 'Fri_date'] ).size().to_frame(name = 'count').reset_index()
grouped_df = grouped_df.groupby(['POD', 'Fri_date']).size().to_frame(name = 'count').reset_index()
grouped_df

In [ ]:
final_cont_df = pd.DataFrame()
for i, row in grouped_df.iterrows():
    pod_value=grouped_df.get_value(i,'POD',takeable=False)
    fri_date_value=grouped_df.get_value(i,'Fri_date',takeable=False)
    qty_value=grouped_df.get_value(i,'count',takeable=False)
    if fri_date_value == start_date:
        low_limit=fri_date_value-timedelta(days=500)
    else:
        low_limit=fri_date_value-timedelta(days=6)
    sub_pd = cont_hdr_pd[(cont_hdr_pd['POD'] == pod_value) & 
                         (cont_hdr_pd['ETA_Date'] <= fri_date_value)& 
                         (cont_hdr_pd['ETA_Date'] >= low_limit)]
    sub_pd = sub_pd.sort_values('ETA_Date',ascending=True)
    qtysum=0
    finaldatelst=[]
    if fri_date_value <= start_date:
        limit = 1
    elif pod_value == 'WILMINGTON':
        limit = 0.5
    else:
        limit = 0.7
    
    for j, rowj in sub_pd.iterrows():
        qtysum=qtysum + 1
        percent = float(qtysum/qty_value)
        if percent<=limit:
            final_date=fri_date_value
        else:
            final_date=fri_date_value + timedelta(days=7)
        finaldatelst.append(final_date)
    se_sub = pd.Series(finaldatelst)    
    sub_pd['Final_date'] = se_sub.values
    final_cont_df = final_cont_df.append(sub_pd)
final_cont_df.reset_index(inplace=True)
grouped_final_df = final_cont_df.groupby( [ 'Logical Tracking ID','POD', 'Final_date'] ).size().to_frame(name = 'count').reset_index()
grouped_final_df = final_cont_df.groupby(['POD', 'Final_date']).size().to_frame(name = 'count').reset_index()

final_cont_df.to_csv(r'\\hftdata02\sysdata02\RedCat\python\ETA_Adjustment\cont_hdr_adjusted.csv', sep=',',index=False)

In [ ]:
grouped_df = final_cont_df.groupby( [ 'POD', 'Final_date'] ).size().to_frame(name = 'count').reset_index()
grouped_df

In [ ]:
final_cont_df = final_cont_df.set_index('Logical Tracking ID')
cont_pd = cont_dtl_pd.join(final_cont_df, rsuffix='_r')
cont_pd['Track_ID'] = cont_pd['PO Number']+'-'+cont_pd.index
cont_pd.reset_index(inplace=True)
cont_pd['Ship Qty'] = cont_pd[cont_pd['Ship Qty'] > 0]['Ship Qty']
cont_pd = cont_pd[np.isfinite(cont_pd['Ship Qty'])]
po_pd = po_pd[['SKU', 'PO Number', 'Adj_Qty', 'Ship To Location',       
       'Shipped Qty', 'Approved_Date', 'Vendor No', 'FOB-Cost', 'WE_Date']]
po_pd['POD']='NOT SHIPPED'
po_pd=po_pd.rename(columns={'Adj_Qty':'PO Open Qty'})
po_pd.head()

In [ ]:
len(cont_pd)

In [ ]:
cont_pd['Ship To Location']='DILLON'
cont_pd.loc[cont_pd['Org']=='MOR','Ship To Location']='MORENO VALLEY'
cont_pd.loc[cont_pd['Org']=='CAM','Ship To Location']='CAMARILLO'
#cont_pd.head()

In [ ]:
#Filters out all of the lines that were received since the file was last updated. 
cont_pd = cont_pd[~(cont_pd['Track_ID'].isin(wkend_pd['Track_ID']))]
len(cont_pd)

In [ ]:
cont_pd=cont_pd[['Item','PO Number','Ord Qty', 'Ship To Location','Ship Qty','ETD_Date','Supplier Number','POD','Final_date']]
cont_pd = cont_pd.rename(columns={'Item':'SKU','PO Number':'PO Number','Ord Qty':'PO Open Qty', 'Ship To Location':'Ship To Location',
                                  'POD':'POD','Ship Qty':'Shipped Qty','ETD_Date':'Approved_Date','Supplier Number':'Vendor No',
                                  'Final_date':'ETA_Date'})

In [ ]:
cont_pd['FOB-Cost']=0
cont_pd['WE_Date']=cont_pd['ETA_Date'] + timedelta(days=7)
cont_pd['PO Open Qty'] = cont_pd['Shipped Qty'].values
cont_pd['ID'] = cont_pd['SKU'].map(str)+'-'+cont_pd['PO Number'].map(str)
cont_pd.head()

In [ ]:
cont_pd.to_csv(r'\\hftdata02\sysdata02\RedCat\python\ETA_Adjustment\cont_pd_adjusted.csv', sep=',',index=False)

In [ ]:
grouped_df = cont_pd.groupby( [ 'POD', 'ETA_Date'] ).size().to_frame(name = 'count').reset_index()
#grouped_df

In [ ]:
inbound_pd = po_pd.append(cont_pd)
#inbound_pd.head()

In [ ]:
inbound_pd.head()

In [ ]:
po_pd.head()

### Correct for partial Receipts

In [ ]:
'''Here is where you will compare the raw PO Out with the new inbound_pd groupbys. '''

In [ ]:
po_check_pd = po_pd
po_check_pd.reset_index(inplace=True)
po_check_pd = po_check_pd[['ID','SKU', 'PO Number', 'PO Open Qty']]
po_check_pd = po_check_pd.groupby(['ID','SKU','PO Number']).sum()
po_check_pd['PO Open Qty'] = po_check_pd[po_check_pd['PO Open Qty'] > 0]['PO Open Qty']
po_check_pd = po_check_pd[np.isfinite(po_check_pd['PO Open Qty'])]
po_check_pd.reset_index(inplace=True)
po_check_pd.set_index('ID', inplace=True)

for i, row in po_check_pd.iterrows():
    open_value=po_check_pd.get_value(i,'PO Open Qty',takeable=False)
    sku_value=po_check_pd.get_value(i,'SKU',takeable=False)
    num_value=po_check_pd.get_value(i,'PO Number',takeable=False)
po_check_pd.head()

In [ ]:
open_po_out_value=po_check_pd.get_value(i,'PO Open Qty',takeable=False)
open_po_out_value

In [ ]:
inbound_check = inbound_pd[(inbound_pd['PO Number']==num_value) & (inbound_pd['SKU']==sku_value)]
#inbound_check = inbound_pd['SKU']==sku_value
inbound_check

In [ ]:
opens = []
open_qty = 0
total_open = 0
for ib, row in inbound_check.iterrows():
    open_py_value = inbound_check.get_value(i,'PO Open Qty',takeable=False)
    opens.append(open_qty)
    total_open = total_open + open_qty
if open_py_value/open_po_out_value == 1:
    print('all good in the hood')
else:
    print('no go joe')

### Final output preperations

In [ ]:
po_lst = ['PO_01','PO_02','PO_03','PO_04','PO_05','PO_06','PO_07','PO_08','PO_09',
          'PO_10','PO_11','PO_12','PO_13','PO_14','PO_15','PO_16','PO_17','PO_18','PO_19',
          'PO_20','PO_21','PO_22','PO_23','PO_24','PO_25','PO_26','PO_27','PO_28','PO_29',
          'PO_30','PO_31','PO_32','PO_33','PO_34','PO_35','PO_36','PO_37','PO_38','PO_39',
          'PO_40','PO_41','PO_42','PO_43','PO_44','PO_45','PO_46','PO_47','PO_48','PO_49',
          'PO_50','PO_51','PO_52']

po_no_lst = ['PO_NO_01','PO_NO_02','PO_NO_03','PO_NO_04','PO_NO_05','PO_NO_06','PO_NO_07','PO_NO_08','PO_NO_09',
             'PO_NO_10','PO_NO_11','PO_NO_12','PO_NO_13','PO_NO_14','PO_NO_15','PO_NO_16','PO_NO_17','PO_NO_18','PO_NO_19',
             'PO_NO_20','PO_NO_21','PO_NO_22','PO_NO_23','PO_NO_24','PO_NO_25','PO_NO_26','PO_NO_27','PO_NO_28','PO_NO_29',
             'PO_NO_30','PO_NO_31','PO_NO_32','PO_NO_33','PO_NO_34','PO_NO_35','PO_NO_36','PO_NO_37','PO_NO_38','PO_NO_39',
             'PO_NO_40','PO_NO_41','PO_NO_42','PO_NO_43','PO_NO_44','PO_NO_45','PO_NO_46','PO_NO_47','PO_NO_48','PO_NO_49',
             'PO_NO_50','PO_NO_51','PO_NO_52']
x = 0
for i in po_lst:
    x = x+1
    if x ==1:
        inbound_pd[i] = np.where(inbound_pd['WE_Date']<=start_date + timedelta(days=7*x),inbound_pd['PO Open Qty'],0)
    else:
        inbound_pd[i] = np.where(inbound_pd['WE_Date']==start_date + timedelta(days=7*x),inbound_pd['PO Open Qty'],0)
x = 0
for i in po_no_lst:
    x = x+1
    if x ==1:
        inbound_pd[i] = np.where(inbound_pd['WE_Date']<=start_date + timedelta(days=7*x),inbound_pd['PO Number'],0)
    else:
        inbound_pd[i] = np.where(inbound_pd['WE_Date']==start_date + timedelta(days=7*x),inbound_pd['PO Number'],0)

In [ ]:
writer = pd.ExcelWriter('Inbound_consolidated_data.xlsx', engine='xlsxwriter')
inbound_pd.to_excel(writer,'inbound_pd')

In [ ]:
test=pd.DataFrame()

In [ ]:
import pandas as pd

test=pd.DataFrame()
test.reset_index()
test.set_index(['Sku','Date'], inplace=True)